<a href="https://colab.research.google.com/github/PiyushChall/SpamClassifier/blob/main/Spam_HamClassificationUsingNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SPAM/HAM Classification using NaturalLanguageProcessing(NLP).**

# Required installation if executing on collab

In [1]:
!pip install nltk
!pip install autocorrect
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=84c5eaf7941db9e8596e6f984ec357c8c04d292ecd4584e1a3821dae638f6c9e
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


# Importing Required modules

In [2]:
import string
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize as wt
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.corpus import stopwords
from tqdm import tqdm
from autocorrect import Speller
spell = Speller()
import re
import matplotlib.pyplot as plt

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score

# Selecting Model [Naive Bayes (Gaussian Naive Bayes)]

In [5]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

# Reading Data and creating a Dataframe

In [6]:
data = pd.read_csv('spam.csv',encoding='latin') #Reading the Dataset
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Deleting unwanted columns

In [7]:
data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace = True) #Droping the unwanted columns

In [8]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Renaming our columns

In [9]:
data.rename(columns={'v1':'Lable','v2':'mail_body'},inplace = True) #Changing the name of the columns

In [10]:
data.head()

,Lable,mail_body
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Giving 1 as a value for Spam
and 0 as a value for Ham

In [11]:
data['Lable'] = data['Lable'].apply(lambda x: 1 if x=='spam' else 0) #Changing 'SPAM' to 1 and 'HAM' to 0

In [12]:
data.head()

,Lable,mail_body
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


# Generating List of Stopwords

In [13]:
stop = stopwords.words('english') #Set of Stopwords

In [14]:
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

# Language Processing

In [15]:
new_data = []
for i in tqdm(range(data.shape[0])):
  lines = data.iloc[i, 1]
  new_lines = re.sub('[^A-Za-z]', ' ', lines).lower()
  tokenized_lines = wt(new_lines)
  cleaned_lines = []
  for word in tokenized_lines:
    if word not in stop:
      cleaned_lines.append(spell(stemmer.stem(word)))
  processed_lines = ' '.join(cleaned_lines)
  new_data.append(processed_lines)

100%|██████████| 5572/5572 [03:08<00:00, 29.61it/s]


In [16]:
new_data

['go during point crazy avail bugs n great world la e buffet cine got amor wat',
 'ok lar joke if u oni',
 'free entry wki comp win fa cup final tt st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf live around though',
 'frees hey dark week word back like fun still tb ok xxx std chg send rev',
 'even brother like speak treat like aid patent',
 'per request well well oru minnaminungint nurungu vietnam set callertun caller press copy friend callertun',
 'winner value network custom select received prize reward claim call claim code kl valid hour',
 'mobil month u r entity update latest colour mobil camera free call mobil update co free',
 'gon na home soon want talk stuff anymore tonight k cri enough today',
 'six chance win cash pound txt cs send cost p day day stand apply reply hl info',
 'urgent week free membership prize jackpot txt word claim c www duk net lccltd box link rw',
 'search right word thank breathe pro

# Organizing Data for Test and Train

In [17]:
x = new_data

In [18]:
x

['go during point crazy avail bugs n great world la e buffet cine got amor wat',
 'ok lar joke if u oni',
 'free entry wki comp win fa cup final tt st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf live around though',
 'frees hey dark week word back like fun still tb ok xxx std chg send rev',
 'even brother like speak treat like aid patent',
 'per request well well oru minnaminungint nurungu vietnam set callertun caller press copy friend callertun',
 'winner value network custom select received prize reward claim call claim code kl valid hour',
 'mobil month u r entity update latest colour mobil camera free call mobil update co free',
 'gon na home soon want talk stuff anymore tonight k cri enough today',
 'six chance win cash pound txt cs send cost p day day stand apply reply hl info',
 'urgent week free membership prize jackpot txt word claim c www duk net lccltd box link rw',
 'search right word thank breathe pro

In [19]:
y=data['Lable']

In [20]:
y # 0 is Ham and 1 is Spam

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: Lable, Length: 5572, dtype: int64

# Splitting the data for Testing and Training

In [21]:
X_train,X_test,Y_train,Y_test=tts(x,y,test_size = 0.25)

In [22]:
X_train

['thank princess want see nice juice boot',
 'gon na home soon want talk stuff anymore tonight k cri enough today',
 'drive read need write',
 'whenever ur sad whenever ur gray remember im list water u wan na say ju walk wid little amp promise bring back ur smile',
 'pain urn thing els',
 'buy today need c meh',
 'costa del sol holiday await collect call claim sae tc box stockport sk xh cost pm max min',
 'oh intent bad time great finger train play along give fifteen min warn',
 'cancel yeah baby well sound import understand darwin give ring later font love kate x',
 'new local date area lot new people relist area reply date start www flirtparti us reply',
 'plural noun research',
 'guy bitch act like interest buy some els next week gave us free',
 'might acid tookplac somewhere ghodbandar rd traffic move love plz slip amp worry',
 'sm service include text credit pl goto www com net login unsubscrib stop extra charg help po box ip',
 'home lei',
 'bill that bad',
 'thought see',
 'alre

In [23]:
len(X_train)

4179

In [24]:
X_test

['log wat dry',
 'hey babe far spun sp da mo dead da world sleep da sofa day',
 'not month need school weed alcohol',
 'ok take ur time n enjoy ur dinner',
 '',
 'tell budget one',
 'urgent tri contact u today draw show prize guarantee call land line claim j po box c pm',
 'need lar go engine co si art today',
 'ok tell half hr b u come need prepare',
 'wil reach',
 'sorry took long mw',
 'house water dock boat roll newscast label jazz flute behind wheel',
 'u send home first ok lor read yet',
 'dizzamn right ask suite get back',
 'yli tampa sarasota',
 'way home long doi spell season would',
 'ill call u mw finish address ichi american free wont stop calling bad jen k eh',
 'r home come within min',
 'didnt get any da',
 'free limit hardcore porn direct mobil txt porn get free access hr child p per day txt stop exit msg free',
 'k must book huh go yoga basic sunday',
 'told hr want post chennai work',
 'india go draw semi mani year south african soil',
 'text get call phone problem',


In [25]:
len(X_test)

1393

In [26]:
Y_train

5245    0
10      0
4993    0
1315    0
427     0
       ..
2702    0
798     0
5538    0
1660    0
2749    0
Name: Lable, Length: 4179, dtype: int64

In [27]:
len(Y_train)

4179

# Vectorizing the data so that the machine can easily process it

In [28]:
X_train_vectorised = cv.fit_transform(X_train).toarray() #Vectorizing X_train
X_test_vectorised = cv.transform(X_test).toarray() #Vectorizing X_test

In [29]:
X_train_vectorised

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [30]:
X_test_vectorised

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Training Model

In [31]:
model

MultinomialNB()

In [32]:
model.fit(X_train_vectorised,Y_train) #Training the Model

MultinomialNB()

Our model's prediction

In [33]:
Y_pred = model.predict(X_test_vectorised)

Accuracy of our model

In [34]:
Accuracy = accuracy_score(Y_test,Y_pred) * 100

In [35]:
Accuracy

97.91816223977028

Confusion matrix of our matrix

In [36]:
CM = confusion_matrix(Y_test,Y_pred)

In [37]:
CM

array([[1203,   14],
       [  15,  161]])

# Generating precission, recall, fscore and support

In [38]:
precision, recall, fscore, support = score(Y_test, Y_pred, pos_label = 1,average='binary')

In [39]:
Precision =np.round(precision,3)

In [40]:
Recall =np.round(recall,3)

In [41]:
print("The Precision of the model is : ",Precision)
print("The Recall of the model is : ",Recall)
print("The Accuracy of the model is : ",Accuracy)
print("The Confusion Matrix of the model is : \n",CM)

The Precision of the model is :  0.92
The Recall of the model is :  0.915
The Accuracy of the model is :  97.91816223977028
The Confusion Matrix of the model is : 
 [[1203   14]
 [  15  161]]
